## python-dotenv

In [2]:
from dotenv import load_dotenv, find_dotenv
import os


load_dotenv(find_dotenv(), override=True)

True

## ChatModels

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)

response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [6]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print(response.content)

The capital of France is Paris!


## Caching LLM Response
#### In-memory Cache

In [8]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
prompt = "What is the capital of France?"
response = llm.invoke(prompt)
print(response.content)

The capital of France is Paris.
CPU times: total: 15.6 ms
Wall time: 299 ms


In [9]:
%%time
response = llm.invoke(prompt)
print(response.content)

The capital of France is Paris.
CPU times: total: 0 ns
Wall time: 743 μs


## LLM Streaming

In [13]:
prompt = "Write a poem about haiku?"

for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Haiku's gentle art
Syllables in perfect part
Nature's fleeting heart

In three lines, a tale is told
Of seasons, sun, and snow cold
Moments to behold

The traditional form
Five-seven-five, a wistful norm
Concise, yet profound

A glimpse of life's brief span
A haiku captures the plan
Of nature's subtle hand

In haiku's quiet space
The world's beauty finds its place
A moment's peaceful face

The art of haiku's skill
To capture life's fleeting will
In few, well-chosen words still

A haiku's beauty lies
In its simplicity's guise
A reflection of the skies